In [ ]:
!pip install google-generativeai


"""
Implementación del Asistente de IA para Clasificación de Gastos
"""

import google.generativeai as genai
import json
import os

genai.configure(api_key=API_KEY)

# 2. DEFINICIÓN DE LA BASE DE CONOCIMIENTO

knowledge_base = {
    "categorias_gasto": [
        {"codigo": "IT-HW-001", "descripcion": "Hardware de Tecnologías de la Información. Incluye laptops, desktops, servidores, monitores y periféricos como impresoras o teclados."},
        {"codigo": "IT-SW-002", "descripcion": "Software y Licencias de TI. Incluye suscripciones a servicios en la nube (SaaS), licencias de software de productividad y sistemas operativos."},
        {"codigo": "MKT-ADV-003", "descripcion": "Publicidad y Marketing Digital. Incluye gastos en campañas de Google Ads, Meta Ads y publicidad en redes sociales."},
        {"codigo": "OFC-SPL-004", "descripcion": "Suministros de Oficina. Incluye material de papelería, cartuchos de tinta, resmas de papel y otros consumibles de oficina."},
        {"codigo": "TRV-EXP-005", "descripcion": "Gastos de Viaje. Incluye boletos de avión, alojamiento en hoteles, y transporte terrestre para viajes de negocio."}
    ],
    "ejemplos_part_numbers": [
        {"part_number": "DELL-LAT-7490", "descripcion": "Laptop Dell Latitude 7490, 16GB RAM, 512GB SSD", "codigo_correcto": "IT-HW-001"},
        {"part_number": "MS-O365-E3", "descripcion": "Licencia anual Microsoft Office 365 Enterprise E3", "codigo_correcto": "IT-SW-002"},
        {"part_number": "FB-ADS-Q3", "descripcion": "Campaña de publicidad en Facebook para el tercer trimestre", "codigo_correcto": "MKT-ADV-003"},
        {"part_number": "STAP-PPR-500", "descripcion": "Resma de papel de 500 hojas tamaño carta", "codigo_correcto": "OFC-SPL-004"}
    ]
}


# 3. FUNCIÓN PARA CREAR EL PROMPT DINÁMICO
def crear_prompt(item_a_clasificar, base_conocimiento):


    categorias_str = "\n".join([f"- Código: {c['codigo']}, Descripción: {c['descripcion']}" for c in base_conocimiento['categorias_gasto']])
    ejemplos_str = "\n".join([f"- Descripción: \"{e['descripcion']}\" -> Código: {e['codigo_correcto']}" for e in base_conocimiento['ejemplos_part_numbers']])


    prompt = f"""
    **Rol:** Eres un experto analista financiero especializado en la clasificación de gastos (Spend Classification). Tu tarea es asignar el código de gasto correcto a un item basándote en su descripción.

    **Contexto - Catálogo de Códigos de Gasto:**
    A continuación se presentan las categorías de gasto disponibles. Analiza cuidadosamente la descripción de cada una:
    {categorias_str}

    **Ejemplos de Clasificaciones Correctas (Few-Shot Learning):**
    Usa los siguientes ejemplos para entender el patrón de clasificación:
    {ejemplos_str}

    **Tarea:**
    Ahora, clasifica el siguiente item. Quiero que sigas un proceso de razonamiento paso a paso (Chain-of-Thought) para llegar a la conclusión.

    **Item a clasificar:** "{item_a_clasificar}"


    return prompt


# 4. FUNCIÓN PRINCIPAL PARA CLASIFICAR EL GASTO (CON LLAMADA REAL A LA API)
def clasificar_gasto(descripcion_item, base_conocimiento, modelo):


    prompt_completo = crear_prompt(descripcion_item, base_conocimiento)

    try:
        response = modelo.generate_content(prompt_completo)

        cleaned_response_text = response.text.strip().replace("```json", "").replace("```", "")

        resultado_json = json.loads(cleaned_response_text)
        return resultado_json

    except json.JSONDecodeError:

        print(f"Error: La respuesta del modelo no pudo ser decodificada como JSON.")
        print(f"Respuesta recibida:\n---\n{response.text}\n---")
        return {
            "codigo_asignado": "ERROR_JSON",
            "razonamiento": "El modelo devolvió una respuesta con formato incorrecto."
        }
    except Exception as e:

        print(f"Ha ocurrido un error inesperado: {e}")
        return {
            "codigo_asignado": "ERROR_API",
            "razonamiento": f"Error al comunicarse con la API: {str(e)}"
        }

# 5. EJECUCIÓN Y DEMOSTRACIÓN
if __name__ == '__main__':

    model = genai.GenerativeModel('gemini-pro')

    items_de_prueba = [
        "Impresora HP LaserJet Pro M404dn",
        "Suscripción mensual a Salesforce CRM",
        "Campaña publicitaria en Google para lanzamiento de producto",
        "Cartucho de tinta negra HP 664",
        "Pasaje aéreo a Madrid para conferencia"
    ]

    print("--- Iniciando Clasificación Automática de Gastos ---")

    for item in items_de_prueba:
        print("-" * 40)
        print(f"▶️  Item a clasificar: {item}")
        
        resultado = clasificar_gasto(item, knowledge_base, model)
        
        print(f"✅ Código Asignado: {resultado.get('codigo_asignado', 'N/A')}")
        print(f"🧠 Razonamiento: {resultado.get('razonamiento', 'N/A')}")

    print("\n--- Clasificación Finalizada ---")